Initialize "property_tax" table before data load

In [0]:
CREATE TABLE IF NOT EXISTS "property_tax" (
    PID                     VARCHAR(255) NOT NULL,          -- Assuming PID is an identifier
    LEGAL_TYPE              VARCHAR(255),          -- Assuming LEGAL_TYPE is categorical text
    ZONING_DISTRICT         VARCHAR(255),          -- Text describing zoning district
    ZONING_CLASSIFICATION   VARCHAR(255),          -- Text describing zoning classification
    TO_CIVIC_NUMBER         VARCHAR(255),          -- Civic number, could be text or number
    STREET_NAME             VARCHAR(255),          -- Street name as text
    PROPERTY_POSTAL_CODE    VARCHAR(20) NOT NULL,  -- Postal code as text
    CURRENT_LAND_VALUE      DECIMAL(18,2),         -- Monetary value, supports decimals
    CURRENT_IMPROVEMENT_VALUE DECIMAL(18,2),       -- Monetary value, supports decimals
    TAX_ASSESSMENT_YEAR     INT,                   -- Integer year
    PREVIOUS_LAND_VALUE     DECIMAL(18,2),         -- Monetary value, supports decimals
    PREVIOUS_IMPROVEMENT_VALUE DECIMAL(18,2),      -- Monetary value, supports decimals
    YEAR_BUILT              INT,                   -- Year, integer
    BIG_IMPROVEMENT_YEAR    INT,                   -- Year, integer
    TAX_LEVY                DECIMAL(18,2),         -- Monetary value, supports decimals
    NEIGHBOURHOOD_CODE      VARCHAR(255),          -- Text or code for neighborhood
    REPORT_YEAR             INT NOT NULL,                    -- Year, integer
    PRIMARY KEY(PROPERTY_POSTAL_CODE, PID)
);

SELECT * 
FROM property_tax
LIMIT 10;

SELECT COUNT(*) AS total_entries
FROM property_tax;

Initialize "building_permits" table before data load

In [0]:
CREATE TABLE IF NOT EXISTS "building_permits" (
    ISSUE_DATE DATE,
    PROJECT_VALUE DECIMAL(15, 2),
    TYPE_OF_WORK VARCHAR(255),
    ADDRESS VARCHAR(255),
    POSTAL_CODE VARCHAR(20),
    PERMIT_CATEGORY VARCHAR(255),
    PROPERTY_USE VARCHAR(255),
    SPECIFIC_USE_CATEGORY TEXT,
    ISSUE_YEAR INT,
    YEAR_MONTH VARCHAR(7),
    GEOM TEXT,
    GEO_POINT_2D TEXT,
    GEO_LOCAL_AREA VARCHAR(255)
);

SELECT * 
FROM building_permits
LIMIT 10;

SELECT COUNT(*) AS total_entries
FROM building_permits;

Generate a postal code matching table for local area

In [0]:
DROP VIEW IF EXISTS "postal_code_local_area_view" CASCADE;

CREATE VIEW "postal_code_local_area_view" AS
SELECT
    p.PROPERTY_POSTAL_CODE AS POSTAL_CODE,
    b.GEO_LOCAL_AREA AS LOCAL_AREA
FROM
    "property_tax" p
JOIN
    "building_permits" b
    ON p.PROPERTY_POSTAL_CODE = b.POSTAL_CODE;

SELECT * FROM "postal_code_local_area_view"
LIMIT 10;

SELECT COUNT(*) AS total_rows
FROM "postal_code_local_area_view";

Group property tax by report year and local area

In [0]:
-- Drop the view if it exists
DROP VIEW IF EXISTS "property_tax_by_report_year_local_area_view" CASCADE;

-- Create the view to group property tax by report year and local area
CREATE VIEW "property_tax_by_report_year_local_area_view" AS
SELECT
    pt.REPORT_YEAR AS YEAR,
    pla.LOCAL_AREA,
    AVG(pt.CURRENT_LAND_VALUE) AS average_land_value,
    AVG(pt.CURRENT_IMPROVEMENT_VALUE) AS average_improvement_value,
    AVG(pt.TAX_LEVY) AS average_tax_levy,
    -- Sum of residential zoning classifications
    SUM(CASE 
            WHEN pt.ZONING_CLASSIFICATION IN ('One-Family Dwelling', 'Multiple Dwelling', 'Residential', 
                                              'Residential Inclusive', 'Two-Family Dwelling', 'Residential Rental') 
            THEN 1 
            ELSE 0 
        END) AS residential_zoning_count,
    -- Sum of non-residential zoning classifications
    SUM(CASE 
            WHEN pt.ZONING_CLASSIFICATION IN ('Commercial', 'Industrial', 'Historical Area', 
                                              'Limited Agriculture', 'Other', 'Comprehensive Development') 
            THEN 1 
            ELSE 0 
        END) AS non_residential_zoning_count
FROM
    "property_tax" pt
JOIN
    "postal_code_local_area_view" pla
    ON pt.PROPERTY_POSTAL_CODE = pla.POSTAL_CODE
GROUP BY
    pt.REPORT_YEAR, pla.LOCAL_AREA;


-- Auxiliary SQL statement to select top 10 rows from the view
SELECT * FROM "property_tax_by_report_year_local_area_view"
LIMIT 10;

-- Auxiliary SQL statement to count how many rows are in the view
SELECT COUNT(*) AS total_rows
FROM "property_tax_by_report_year_local_area_view";

Group building permits by issued year and local area

In [0]:
-- Drop the view if it exists
DROP VIEW IF EXISTS "building_permits_by_year_local_area_view" CASCADE;

-- Create the view to group building permits by issued year and local area
CREATE VIEW "building_permits_by_year_local_area_view" AS
SELECT
    bp.ISSUE_YEAR AS YEAR,
    bp.GEO_LOCAL_AREA AS LOCAL_AREA,
    COUNT(*) AS total_permits_issued,
    AVG(bp.PROJECT_VALUE) AS average_project_value,
    -- Sum of residential property uses
    SUM(CASE 
            WHEN bp.PROPERTY_USE IN ('Dwelling Uses', 'Live-Work Uses') 
            THEN 1 
            ELSE 0 
        END) AS residential_property_use_count,
    -- Sum of non-residential property uses
    SUM(CASE 
            WHEN bp.PROPERTY_USE IN ('Office Uses', 'Retail Uses', 'Service Uses', 'Institutional Uses', 
                                     'Cultural/Recreational Uses', 'Manufacturing Uses', 'Parking Uses', 
                                     'Wholesale Uses', 'Transportation and Storage Uses', 'Utility and Communication Uses', 
                                     'Agricultural Uses', 'Deposition or Extraction Uses', 'Mural') 
            THEN 1 
            ELSE 0 
        END) AS non_residential_property_use_count
FROM
    "building_permits" bp
GROUP BY
    bp.ISSUE_YEAR, bp.GEO_LOCAL_AREA;

-- Auxiliary SQL statement to select top 10 rows from the view
SELECT * FROM "building_permits_by_year_local_area_view"
LIMIT 10;

-- Auxiliary SQL statement to count how many rows are in the view
SELECT COUNT(*) AS total_rows
FROM "building_permits_by_year_local_area_view";

Join two grouped views by year and local area and unload

In [0]:
-- Drop the view if it exists
DROP VIEW IF EXISTS "combined_building_permits_property_tax_view" CASCADE;

-- Create the view to join building permits and property tax by year and local area
CREATE VIEW "combined_building_permits_property_tax_view" AS
SELECT
    bp.YEAR,
    bp.LOCAL_AREA,
    bp.total_permits_issued,
    bp.average_project_value,
    bp.residential_property_use_count,
    bp.non_residential_property_use_count,
    pt.average_land_value,
    pt.average_improvement_value,
    pt.average_tax_levy,
    pt.residential_zoning_count,
    pt.non_residential_zoning_count
FROM
    "building_permits_by_year_local_area_view" bp
JOIN
    "property_tax_by_report_year_local_area_view" pt
    ON bp.YEAR = pt.YEAR
    AND bp.LOCAL_AREA = pt.LOCAL_AREA;

-- Auxiliary SQL statement to select top 10 rows from the view
SELECT * FROM "combined_building_permits_property_tax_view";

-- Auxiliary SQL statement to count how many rows are in the view
SELECT COUNT(*) AS total_rows
FROM "combined_building_permits_property_tax_view";

In [0]:
-- Unload final results to output storage in s3
UNLOAD ('SELECT * FROM combined_building_permits_property_tax_view')
TO 's3://cmpt732-project-raw-data/analysis_output/'
IAM_ROLE 'arn:aws:iam::354918385749:role/AWSRedshiftOutputRole'
DELIMITER ','
HEADER
ALLOWOVERWRITE
PARALLEL OFF;

Unload filtered building_permits for use in visualization

In [0]:
-- Unload clean tables to output storage in s3
UNLOAD ('SELECT * FROM building_permits')
TO 's3://cmpt732-project-raw-data/analysis_output/'
IAM_ROLE 'arn:aws:iam::354918385749:role/AWSRedshiftOutputRole'
DELIMITER ';'
HEADER
ALLOWOVERWRITE
PARALLEL OFF;

Unload filtered property_tax for use in visualization

In [0]:
-- Unload clean tables to output storage in s3
UNLOAD ('SELECT * FROM property_tax')
TO 's3://cmpt732-project-raw-data/analysis_output/'
IAM_ROLE 'arn:aws:iam::354918385749:role/AWSRedshiftOutputRole'
DELIMITER ';'
HEADER
ALLOWOVERWRITE
PARALLEL OFF;

Group property_tax by postal

In [0]:
-- Drop the view if it exists
DROP VIEW IF EXISTS "property_tax_grouped" CASCADE;

-- Step 1: Create the view for grouped property tax data by postal code
CREATE OR REPLACE VIEW property_tax_grouped AS
SELECT 
    PROPERTY_POSTAL_CODE,
    COUNT(PID) AS property_count,
    CASE
        -- Residential zones
        WHEN MAX(ZONING_CLASSIFICATION) IN ('One-Family Dwelling', 'Residential Inclusive', 'Residential', 'Two-Family Dwelling', 'Residential Rental') THEN 'residential'
        
        -- Comprehensive (Mixed) zone
        WHEN MAX(ZONING_CLASSIFICATION) = 'Comprehensive Development' THEN 'mixed'
        
        -- Non-residential zones
        WHEN MAX(ZONING_CLASSIFICATION) IN ('Commercial', 'Multiple Dwelling', 'Industrial', 'Historical Area', 'Other', 'Limited Agriculture') THEN 'non-residential'
        
        -- Default case for unknown zoning classifications
        ELSE 'unknown'
    END AS ZONING,
    SUM(CURRENT_LAND_VALUE) AS total_land_value,
    SUM(CURRENT_IMPROVEMENT_VALUE) AS total_improvement_value,
    SUM(CURRENT_LAND_VALUE + CURRENT_IMPROVEMENT_VALUE) AS total_property_value,
    AVG(CURRENT_LAND_VALUE) AS avg_land_value,
    AVG(CURRENT_IMPROVEMENT_VALUE) AS avg_improvement_value,
    AVG(CURRENT_LAND_VALUE + CURRENT_IMPROVEMENT_VALUE) AS avg_property_value,
    MAX(TAX_ASSESSMENT_YEAR) AS latest_assessment_year,
    MAX(YEAR_BUILT) AS latest_year_built,
    MAX(BIG_IMPROVEMENT_YEAR) AS latest_improvement_year
FROM property_tax
GROUP BY PROPERTY_POSTAL_CODE;

-- Step 2: Show the top 10 rows from the grouped property tax view
SELECT * FROM property_tax_grouped
LIMIT 10;

-- Step 3: Get the total number of rows in the grouped property tax view
SELECT COUNT(*) FROM property_tax_grouped;

Group building_permits by postal

In [0]:
-- Drop the view if it exists
DROP VIEW IF EXISTS "building_permits_grouped" CASCADE;

-- Step 1: Create or replace the view for grouped building permits data by postal code
CREATE OR REPLACE VIEW building_permits_grouped AS
SELECT 
    POSTAL_CODE,
    COUNT(*) AS permit_count,
    MAX(GEOM) AS geom,
    MAX(geo_point_2d) AS geo_point_2d,
    -- Classify PROPERTY_USE based on property use
    CASE
        WHEN MAX(PROPERTY_USE) IN ('Dwelling Uses') THEN 'residential'
        ELSE 'non-residential'
    END AS PROPERTY_USE,
    SUM(PROJECT_VALUE) AS total_project_value,
    AVG(PROJECT_VALUE) AS avg_project_value,
    COUNT(DISTINCT TYPE_OF_WORK) AS distinct_work_types,
    COUNT(DISTINCT PERMIT_CATEGORY) AS distinct_permit_categories,
    COUNT(DISTINCT PROPERTY_USE) AS distinct_property_uses,
    MAX(ISSUE_YEAR) AS latest_issue_year,
    MAX(ISSUE_DATE) AS latest_issue_date
FROM building_permits
GROUP BY POSTAL_CODE;

-- Step 2: Show the top 10 rows from the grouped building permits view
SELECT * FROM building_permits_grouped
LIMIT 10;

-- Step 3: Get the total number of rows in the grouped building permits view
SELECT COUNT(*) FROM building_permits_grouped;

Join both by postal code

In [0]:
-- Drop the view if it exists
DROP VIEW IF EXISTS "property_and_permits_grouped" CASCADE;

-- Step 1: Create the view to join the grouped property tax and building permits data by postal code
CREATE OR REPLACE VIEW property_and_permits_grouped AS
SELECT 
    ptg.PROPERTY_POSTAL_CODE AS postal_code,
    ptg.zoning AS zoning,
    bp.property_use AS property_use,
    bp.geom AS geom,
    ptg.property_count AS property_count,
    bp.permit_count AS project_count,
    ptg.total_land_value,
    ptg.total_improvement_value,
    ptg.total_property_value,
    ptg.avg_land_value,
    ptg.avg_improvement_value,
    ptg.avg_property_value,
    bp.permit_count AS permit_count,
    bp.total_project_value,
    bp.avg_project_value,
    bp.latest_issue_year AS year,
    -- Extract latitude and longitude, trim whitespace, cast to double precision, and handle malformed data
    CASE
        WHEN bp.geo_point_2d IS NULL OR 
             LENGTH(bp.geo_point_2d) - LENGTH(REPLACE(bp.geo_point_2d, ',', '')) <> 1 -- Ensure there's exactly one comma
        THEN NULL -- Fallback to NULL if the data is malformed
        ELSE CAST(TRIM(SPLIT_PART(bp.geo_point_2d, ',', 1)) AS DOUBLE PRECISION)
    END AS latitude,
    
    CASE
        WHEN bp.geo_point_2d IS NULL OR 
             LENGTH(bp.geo_point_2d) - LENGTH(REPLACE(bp.geo_point_2d, ',', '')) <> 1 -- Ensure there's exactly one comma
        THEN NULL -- Fallback to NULL if the data is malformed
        ELSE CAST(TRIM(SPLIT_PART(bp.geo_point_2d, ',', 2)) AS DOUBLE PRECISION)
    END AS longitude
FROM property_tax_grouped ptg
JOIN building_permits_grouped bp
    ON ptg.PROPERTY_POSTAL_CODE = bp.POSTAL_CODE;

-- Step 2: Show the top 10 rows from the joined view of property tax and building permits data
SELECT * FROM property_and_permits_grouped
LIMIT 10;

-- Step 3: Get the total number of rows in the joined view of property tax and building permits data
SELECT COUNT(*) FROM property_and_permits_grouped;

In [0]:
-- Unload final results to output storage in s3
UNLOAD ('SELECT * FROM property_and_permits_grouped')
TO 's3://cmpt732-project-raw-data/analysis_output/'
IAM_ROLE 'arn:aws:iam::354918385749:role/AWSRedshiftOutputRole'
DELIMITER ';'
HEADER
ALLOWOVERWRITE
PARALLEL OFF;